In [1]:
import pandas as pd

df = pd.read_csv('token_counts.csv')
print(df.shape)
df.head()

(167294, 2)


,token,n
0,kaepernick,87
1,adderall,73
2,reddiquette,73
3,acid,72
4,park,68


In [2]:
import numpy as np
import distance
import time
"""
Algo strawman...

canon = {}
For each token...
    find canon form with lowest edit distance to token
    if edit distance < some thresh:
        add this to canon's children
    elif n >= some other thresh:
        add this as a new canon term

What should edit distance thresh be?
Well lev("camaraderie", "comradery") = 4. 5 for "comrodery".

>>> j1 = 'johansen'
>>> j2 = 'johansson'
>>> j3 = 'johanson'
>>> lev = distance.levenshtein
>>> lev(j1, j2)
2
>>> lev(j1, j2, normalized=True)                                                                                                                                                                      
0.2222222222222222
>>> lev(j1, j3, normalized=True)                                                                                                                                                                      
0.125
>>> lev(j2, j3, normalized=True)                                                                                                                                                                      
0.1111111111111111

camar/comrad norm dist = .3636...


High level note: this doesn't need to be *perfect*. Can always do some manual correction and
fine-tuning after (which might be cheaper than over-optimizing the heuristics).
Also, relatively easy to fine-tune an alphabetized list as long as it's not too long, since groups that
should be merged should be close to each other (will almost always start with same 1 or 2 chars)
"""

# Remove some false positive terms from consideration
blist = {'park', 'acid', 'vong', 'bay', 'syndrome', 'disease', 'effect', 'bay', 'island', 'khan', 'lee'}

# Post-hoc filtering of some spurious matches
blist2 = ('bechamel claire fairley faile fire fairy farrel carfentanyl hansen'
          ' johnson kardashians karazhan bougie loogies roofie loose coozie'
          ' rachel machine machete lachey michel matcha'
          ' mccauley mcdonough mcdonaugh mcdonagh pescatarians etiquette ettiquette'
          ' shavasana shanahan sole tyrion'
         ).split()
blist2 = set(blist2)
blist = blist.union(blist2)
def naive_cluster(df, max_distance=5, max_clusters=1000, norm=False, min_n=2):
    """Return a dictionary where the values are sets of similar word forms. The key is the word
    form in that set with the greatest number of appearances.
    
    norm: whether to normalize edit distance
    min_n: only look at word forms with at least this many occurrences
    max_clusters: once the dict to be returned has at least this many entries, stop adding 
        new keys (but continue iterating through word forms and comparing them to the existing keys)
    """
    canon = {}
    pairs = zip(df.token, df.n)
    for token, n in pairs:
        if n < min_n:
            break
        if not isinstance(token, str):
            print("Warning: got a token without type str: {!r}".format(token))
            continue
        if len(token) <= 2:
            continue
        if token in blist:
            continue
        # Avoid false-positive associations like Kardashian -> Kardashian's
        if token.endswith("'s") and not (token.startswith('alz') or token.startswith('asb') 
                                         or token.startswith('asp')):
            continue
        best = (99, None)
        # Find closest canonical form (could use ilevenshtein - not sure if it's faster)
        for cand in canon.keys():
            lev = distance.levenshtein(token, cand, 
                                       max_dist=max_distance,
                                       normalized=norm
                                      )
            if (not norm and lev != -1) or lev < max_distance:
                if best[0] < 99:
                    # Could try to be really fancy here and do a merge.
                    print("Ambiguous match for token {}. {} has score {}. {} has score {}".format(
                        token, cand, lev, best[1], best[0]
                    ))
                if lev < best[0]:
                    best = (lev, cand)
        if best[0] < 99:
            canon[best[1]][token] = n
        # If we didn't find a match and there's still room, add this token as a new canonical form
        elif len(canon) < max_clusters:
            canon[token] = {token: n}
    return canon

t0 = time.time()
c = naive_cluster(df, max_distance=.37, norm=True, max_clusters=30, min_n=5)
print("Ran in {:.1f}s".format(time.time()-t0))
for canon, forms in sorted(c.items(), key=lambda tup: tup[0]):
    if len(forms) > 1 or True:
        total = sum(forms.values())
        print(canon, ':', forms, 'total =', total)

Ran in 13.5s
adderall : {'adderall': 73, 'aderal': 10, 'adderral': 9, 'aderall': 16, 'adderol': 17, 'adderoll': 6, 'adderal': 46, 'adderrall': 8} total = 185
alzheimers : {'alzheimers': 36, "alzheimer's": 8, 'altzheimers': 6} total = 50
asbergers : {"asperger's": 7, 'asbergers': 36, 'aspbergers': 15, 'asburgers': 9, 'aspergers': 31} total = 98
comradery : {'comeraderie': 8, 'camaraderie': 6, 'comaraderie': 5, 'comraderie': 24, 'comradery': 54, 'cameraderie': 7, 'comradere': 8} total = 112
deschanel : {'dechanel': 8, 'deschutes': 6, 'deshanel': 7, 'deschanel': 48} total = 69
dilaudid : {'dilauded': 23, 'diladid': 5, 'dilaudid': 41, 'delaudid': 5} total = 74
faire : {'fare': 9, 'fair': 13, 'faire': 36} total = 58
fentanyl : {'fentynol': 21, 'fentanol': 9, 'fentenyl': 5, 'phentanyl': 5, 'fentanyl': 42, 'fentynal': 9, 'fentynl': 5} total = 96
galifinakis : {'galafanakis': 8, 'galifianakis': 14, 'galifinakis': 52, 'galifanakis': 7, 'galafinakis': 17} total = 98
gras : {'gra': 5, 'gray': 27,

In [3]:
for k, v in sorted(c.items(), key=lambda tup: sum(tup[1].values()), reverse=True):
    total = sum(v.values())
    print(k, total)

adderall 185
johansen 158
gyllenhal 151
reddiquette 150
kaepernick 146
mcconaughey 121
shamalan 116
comradery 112
sarkeesian 109
sriracha 101
galifinakis 98
asbergers 98
fentanyl 96
seizure 84
dilaudid 74
soleil 73
gras 71
palahniuk 70
minaj 69
deschanel 69
loogie 66
kardashian 58
faire 58
welbutrin 57
heimlich 54
tyson 51
pescatarian 51
alzheimers 50
silmarillion 50
mache 36


In [4]:
assert False

AssertionError: 

In [ ]:
import pickle

with open('clusters.pickle', 'wb') as f:
    pickle.dump(c, f)

In [ ]:
import sklearn.cluster
# https://stats.stackexchange.com/questions/123060/clustering-a-long-list-of-strings-words-into-similarity-groups

# Well, this is shockingly shitty! Even after trying to tune params like preference and damping.
def affprop_cluster(df, prefs=False):
    words = df.token.values
    lev_similarity = -1*np.array([[
        distance.nlevenshtein(w1,w2) 
        for w1 in words] for w2 in words
    ])
    preference = -.15
    if prefs:
        preference = (-1/df.n) * 5
    affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping=0.9, 
                                                  preference=preference,
                                                  verbose=True,
                                                 )
    affprop.fit(lev_similarity)
    
    for cluster_id in np.unique(affprop.labels_)[:50]:
        exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
        cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
        cluster_str = ", ".join(cluster)
        print(" - *%s:* %s" % (exemplar, cluster_str))
        
#affprop_cluster(df.head(100), True)

In [ ]:
words = df.head(5).token.values
sim = -1*np.array([[distance.nlevenshtein(w1,w2) for w1 in words] for w2 in words])
sim

In [ ]:
# StackOverflow example code

words = """I have the following problem at hand: I have a very long list of words, possibly names, surnames, etc. 
I need to cluster this word list, such that similar words, for example words with similar edit (Levenshtein) 
distance appears in the same cluster. For example "algorithm" and "alogrithm" should have high chances to 
appear in the same cluster.

I am well aware of the classical unsupervised clustering methods like k-means clustering, EM clustering in the Pattern Recognition literature. The problem here is that these methods work on points which reside in a vector space. I have words of strings at my hand here. 
It seems that, the question of how to represent strings in a numerical vector space and to calculate "means" of 
string clusters is not sufficiently answered, according to my survey efforts until now. A naive approach to attack 
this problem would be to combine k-Means clustering with Levenshtein distance, but the question still remains 
"How to represent "means" of strings?". There is a weight called as TF-IDF weight, but it seems that it is 
mostly related to the area of "text document" clustering, not for the clustering of single words. It seems that 
there are some special string clustering algorithms existing, like the one at 

My search in this area is going on still, but I wanted to get ideas from here as well. What would you do recommend in this 
case, is anyone aware of any methods for this kind of problem?
""".split() #Replace this line
words = np.asarray(words) #So that indexing with a list will work
lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])

affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping=0.5)
affprop.fit(lev_similarity)
for cluster_id in np.unique(affprop.labels_):
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))